In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  

In [2]:
import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

In [3]:
NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

NUM_CLASSES_REDUCED = 2
nums = (0, 1)
ratio = (100, 1)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

In [4]:
col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
          #   "mean_40", "variance_40",
          #   "mean_50", "variance_50",
             "cap", "normalization", "other"]

rows = []

In [5]:
norm=False

if norm:
    transform=torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[143.8888, 127.1705, 117.5357], std=[69.8313, 64.5137, 66.9933])])
else:
   # transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    transform=None

train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=transform)  


test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=transform)  

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)

    
reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums, transform=transform)

triplet_train_CIFAR10 = class_sampling.ForTripletLoss(reduced_train_CIFAR10, smote=False, transform=transform, num_classes=2)
triplet_ratio_train_CIFAR10 = class_sampling.ForTripletLoss(ratio_train_CIFAR10, smote=False, transform=transform, num_classes=2)

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED, transform=transform)
triplet_smote_train_CIFAR10 = class_sampling.ForTripletLoss(smote_train_CIFAR10, smote=True, transform=transform, num_classes=2)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
targets = ratio_train_CIFAR10.labels 

class_count = np.unique(targets, return_counts=True)[1]
print(class_count)

weight = 1. / class_count

samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)
undersampler_smote = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * 50 * NUM_CLASSES_REDUCED), replacement=False)
weight *= class_count[0]

[5000   50]


In [7]:
train_loader = DataLoader(train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_smote_undersampled = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler_smote)

train_loader_tripletloss = DataLoader(triplet_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_ratio = DataLoader(triplet_ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_smote = DataLoader(triplet_smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [8]:
# to be used in distance capped smote - get average tensor for the entire class 
dataset = train_loader_ratio.dataset
class0 = dataset.images[dataset.labels==0]
class1 = dataset.images[dataset.labels==1]
class0_avg = torch.mean(class0.float(), 0)
class1_avg = torch.mean(class1.float(), 0)
avg_tensors_list = [class0_avg, class1_avg]

In [ ]:
# 2 CLASS normal

momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)


In [47]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []


In [45]:
# 2 CLASS ratio

momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]

learning_rate_aucs = []
learning_rate_train_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                _, train_auc = metric_utils.auc_sigmoid(train_loader_ratio, network) 
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)



Test set: Avg. loss: 0.0076671247482299806, AUC: 0.6110735


Test set: Avg. loss: 0.0020734254121780394, AUC: 0.628587


Test set: Avg. loss: 0.0008975799964501125, AUC: 0.6439400000000001


Test set: Avg. loss: 0.0019357805252075196, AUC: 0.643949


Test set: Avg. loss: 0.0009144600006834705, AUC: 0.675112


Test set: Avg. loss: 0.0020213021636009214, AUC: 0.67089


Test set: Avg. loss: 0.0008470717963236983, AUC: 0.7536120000000001


Test set: Avg. loss: 0.003886705994606018, AUC: 0.514122


Test set: Avg. loss: 0.0015828227996826172, AUC: 0.595936


Test set: Avg. loss: 0.0012174508654245056, AUC: 0.567244


Test set: Avg. loss: 0.0015572131872177125, AUC: 0.6836819999999999


Test set: Avg. loss: 0.0011150310660647874, AUC: 0.643232


Test set: Avg. loss: 0.0014421252608299256, AUC: 0.712404


Test set: Avg. loss: 0.0012044163608905113, AUC: 0.668856


Test set: Avg. loss: 0.01665996742248535, AUC: 0.6862579999999999


Test set: Avg. loss: 0.0015813767313957213, AUC: 0.69056000000


Test set: Avg. loss: 0.0008650489547981484, AUC: 0.670572


Test set: Avg. loss: 0.0021552451848983766, AUC: 0.7147754999999999


Test set: Avg. loss: 0.0008405291248518642, AUC: 0.711684


Test set: Avg. loss: 0.0025969254970550537, AUC: 0.4249035


Test set: Avg. loss: 0.0020958303213119505, AUC: 0.5708455


Test set: Avg. loss: 0.0009637036760992343, AUC: 0.552364


Test set: Avg. loss: 0.0018786900043487548, AUC: 0.6407069999999999


Test set: Avg. loss: 0.0009473569576840589, AUC: 0.6412439999999999


Test set: Avg. loss: 0.0020249083042144776, AUC: 0.679206


Test set: Avg. loss: 0.0008764915754741961, AUC: 0.679316


Test set: Avg. loss: 0.001352885603904724, AUC: 0.4485485


Test set: Avg. loss: 0.0024831844568252563, AUC: 0.42361899999999997


Test set: Avg. loss: 0.0010917736091973758, AUC: 0.361972


Test set: Avg. loss: 0.002164663791656494, AUC: 0.455156


Test set: Avg. loss: 0.0011078405144191025, AUC: 0.398448


Test set: Avg. loss: 0.0021346499919891355, AUC: 0.501624

In [46]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [39]:
# 2 CLASS oversampled

momentum=0
learning_rates = [1e-3, 5e-3, 1e-4]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)


Test set: Avg. loss: 0.0038651072978973387, AUC: 0.40709100000000004


Test set: Avg. loss: 0.0005063888132572174, AUC: 0.8398785


Test set: Avg. loss: 0.0005734625160694122, AUC: 0.8425819999999999


Test set: Avg. loss: 0.0011212489008903504, AUC: 0.841191


Test set: Avg. loss: 0.0008080781102180481, AUC: 0.55231


Test set: Avg. loss: 0.0006464185416698455, AUC: 0.7963709999999999


Test set: Avg. loss: 0.0010616356730461121, AUC: 0.7623549999999999


Test set: Avg. loss: 0.0011121231317520143, AUC: 0.753925


Test set: Avg. loss: 0.0025173540115356447, AUC: 0.5626525


Test set: Avg. loss: 0.0006488811075687408, AUC: 0.6799000000000001


Test set: Avg. loss: 0.000709791898727417, AUC: 0.6942869999999999


Test set: Avg. loss: 0.0009997697472572326, AUC: 0.726058


Test set: Avg. loss: 0.0008281596899032593, AUC: 0.572139


Test set: Avg. loss: 0.0004913568198680878, AUC: 0.8518579999999999


Test set: Avg. loss: 0.0006246580779552459, AUC: 0.8336830000000001


Test set: Avg. los

In [40]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [52]:
# 2 CLASS undersampled

momentum=0
learning_rates = [5e-2, 1e-2, 1e-3, 5e-3, 1e-4]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_undersampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["undersampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006928627490997315, AUC: 0.5610565


Test set: Avg. loss: 0.0006705484390258789, AUC: 0.731087


Test set: Avg. loss: 0.0006308182775974274, AUC: 0.7689


Test set: Avg. loss: 0.0006022096574306488, AUC: 0.792463


Test set: Avg. loss: 0.0006950775682926178, AUC: 0.4702115


Test set: Avg. loss: 0.0006825314462184907, AUC: 0.7292394999999999


Test set: Avg. loss: 0.0006524708569049835, AUC: 0.754748


Test set: Avg. loss: 0.0006467648446559906, AUC: 0.7728849999999999


Test set: Avg. loss: 0.0006955726444721221, AUC: 0.3766905


Test set: Avg. loss: 0.0007012077569961548, AUC: 0.601467


Test set: Avg. loss: 0.0006722729504108429, AUC: 0.7478565


Test set: Avg. loss: 0.0006584925651550293, AUC: 0.764774


Test set: Avg. loss: 0.0006918634176254272, AUC: 0.548633


Test set: Avg. loss: 0.0006640854477882385, AUC: 0.7368910000000001


Test set: Avg. loss: 0.0006435896456241608, AUC: 0.772193


Test set: Avg. loss: 0.0006132080554962159, AUC: 0.805193


Test se


Test set: Avg. loss: 0.000681488573551178, AUC: 0.661619


Test set: Avg. loss: 0.0006757974326610565, AUC: 0.6790075000000001


Test set: Avg. loss: 0.0006896138191223144, AUC: 0.610887


Test set: Avg. loss: 0.0006893069446086884, AUC: 0.613855


Test set: Avg. loss: 0.0006885486543178558, AUC: 0.633282


Test set: Avg. loss: 0.0006888088881969452, AUC: 0.6441379999999999


Test set: Avg. loss: 0.000698607712984085, AUC: 0.47644650000000005


Test set: Avg. loss: 0.0006970592141151428, AUC: 0.5706005000000001


Test set: Avg. loss: 0.0006956733465194702, AUC: 0.6301249999999999


Test set: Avg. loss: 0.0006951669156551361, AUC: 0.6572


Test set: Avg. loss: 0.0006925795376300812, AUC: 0.575475


Test set: Avg. loss: 0.0006951481699943542, AUC: 0.5105139999999999


Test set: Avg. loss: 0.0006970000267028808, AUC: 0.4651495


Test set: Avg. loss: 0.000699210911989212, AUC: 0.424679


Test set: Avg. loss: 0.0006938398480415344, AUC: 0.49060099999999995


Test set: Avg. loss: 0.00069005

In [53]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [54]:
# 2 Class Weighted Loss 

momentum=0
learning_rates = [5e-2, 1e-2, 5e-3, 1e-3, 5e-4]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['pos_weight'] = torch.tensor([weight[1]])


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                _, train_auc = metric_utils.auc_sigmoid(train_loader_ratio, network) 
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["weighted", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006929865777492523, AUC: 0.6035210000000001


Test set: Avg. loss: 0.0006941011548042297, AUC: 0.5


Test set: Avg. loss: 0.01152815311261923, AUC: 0.5


Test set: Avg. loss: 0.0006933517456054688, AUC: 0.5


Test set: Avg. loss: 0.010536302033037242, AUC: 0.5


Test set: Avg. loss: 0.0006934399306774139, AUC: 0.5


Test set: Avg. loss: 0.01047680665950964, AUC: 0.5


Test set: Avg. loss: 0.0006829564273357391, AUC: 0.7606660000000001


Test set: Avg. loss: 0.0007110981047153472, AUC: 0.513034


Test set: Avg. loss: 0.008160775073684089, AUC: 0.4939


Test set: Avg. loss: 0.0006991162598133087, AUC: 0.5


Test set: Avg. loss: 0.009259643141586001, AUC: 0.5


Test set: Avg. loss: 0.0006932158470153809, AUC: 0.5


Test set: Avg. loss: 0.011024211881184342, AUC: 0.5


Test set: Avg. loss: 0.0006900024712085724, AUC: 0.6275345


Test set: Avg. loss: 0.0006032556593418122, AUC: 0.6915875


Test set: Avg. loss: 0.009716101306499822, AUC: 0.67254


Test set: Avg. loss


Test set: Avg. loss: 0.0005637988150119782, AUC: 0.828475


Test set: Avg. loss: 0.00714115304521995, AUC: 0.9039240000000001


Test set: Avg. loss: 0.0005115714669227601, AUC: 0.872054


Test set: Avg. loss: 0.0058704357690150194, AUC: 0.9506


Test set: Avg. loss: 0.0005990622639656067, AUC: 0.854916


Test set: Avg. loss: 0.004999826568188054, AUC: 0.9478759999999999


Test set: Avg. loss: 0.0006893006265163422, AUC: 0.592972


Test set: Avg. loss: 0.0005413131713867187, AUC: 0.838742


Test set: Avg. loss: 0.009401522638774154, AUC: 0.873108


Test set: Avg. loss: 0.0005082505941390992, AUC: 0.8643450000000001


Test set: Avg. loss: 0.007597315765843533, AUC: 0.93034


Test set: Avg. loss: 0.0005349463224411011, AUC: 0.819852


Test set: Avg. loss: 0.005960197472336269, AUC: 0.952184


Test set: Avg. loss: 0.0006917398273944855, AUC: 0.5357335000000001


Test set: Avg. loss: 0.0005320677161216736, AUC: 0.872421


Test set: Avg. loss: 0.007882021895729669, AUC: 0.885336


Test set:


Test set: Avg. loss: 0.0005055390298366547, AUC: 0.849865


Test set: Avg. loss: 0.008350928868397627, AUC: 0.8736079999999999


Test set: Avg. loss: 0.0006963136792182922, AUC: 0.39968349999999997


Test set: Avg. loss: 0.0005984225273132324, AUC: 0.771389


Test set: Avg. loss: 0.009001542176350508, AUC: 0.78562


Test set: Avg. loss: 0.000558992862701416, AUC: 0.8067799999999999


Test set: Avg. loss: 0.008595412568290635, AUC: 0.829488


Test set: Avg. loss: 0.000538530021905899, AUC: 0.82298


Test set: Avg. loss: 0.007287021987509019, AUC: 0.8586119999999999


Test set: Avg. loss: 0.0006968409717082977, AUC: 0.38355749999999994


Test set: Avg. loss: 0.0006436823904514313, AUC: 0.803137


Test set: Avg. loss: 0.01135210377154964, AUC: 0.7909520000000001


Test set: Avg. loss: 0.000583594560623169, AUC: 0.8077380000000001


Test set: Avg. loss: 0.01014432730060993, AUC: 0.820932


Test set: Avg. loss: 0.0005280269980430603, AUC: 0.833491


Test set: Avg. loss: 0.00862787301587586

In [55]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [22]:
# 2 CLASS SMOTE

momentum=0
learning_rates = [1e-3]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = None


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)



Test set: Avg. loss: 0.0009200392961502075, AUC: 0.678816


Test set: Avg. loss: 0.0006507715582847595, AUC: 0.8034624999999999


Test set: Avg. loss: 0.0006657144129276276, AUC: 0.8041109999999999


Test set: Avg. loss: 0.0006028525233268738, AUC: 0.836938


Test set: Avg. loss: 0.0011849600076675416, AUC: 0.401832


Test set: Avg. loss: 0.00063573157787323, AUC: 0.8188884999999999


Test set: Avg. loss: 0.0006494357883930206, AUC: 0.8221820000000001


Test set: Avg. loss: 0.0006822991371154785, AUC: 0.823094


Test set: Avg. loss: 0.00185699599981308, AUC: 0.656679


Test set: Avg. loss: 0.0006367193758487702, AUC: 0.8178314999999999


Test set: Avg. loss: 0.0006523110866546631, AUC: 0.8215329999999998


Test set: Avg. loss: 0.0006271962523460388, AUC: 0.846208


Test set: Avg. loss: 0.003958873510360718, AUC: 0.591901


Test set: Avg. loss: 0.0006755812168121338, AUC: 0.683454


Test set: Avg. loss: 0.0005986700654029846, AUC: 0.836574


Test set: Avg. loss: 0.0006900097727775573, 

In [23]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [ ]:
# 2 Class Capped SMOTE 

momentum=0
learning_rates = [5e-4]


cap_aucs = []

caps = [0]

for cap in caps:
    
    print(cap)
    
    loss_fn_args = {}
    loss_fn_args['print_loss'] = False
    loss_fn_args['print_capped'] = False 
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNet(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                loss_fn_args['loss_cap'] = 10 / ((epoch + 1) / 2)
                _, _ = train.train_sigmoid_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["capped_smote", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm, None]
        rows.append(row)

print(rows)


0

Test set: Avg. loss: 0.0010205329358577728, AUC: 0.7048180000000001


Test set: Avg. loss: 0.0005711885392665863, AUC: 0.8074610000000001


Test set: Avg. loss: 0.0008731485605239868, AUC: 0.7810469999999999


Test set: Avg. loss: 0.000770821899175644, AUC: 0.8042990000000001


Test set: Avg. loss: 0.0053196785449981685, AUC: 0.47757400000000005


Test set: Avg. loss: 0.0005356418192386627, AUC: 0.853467


Test set: Avg. loss: 0.001122176170349121, AUC: 0.833386


Test set: Avg. loss: 0.0011779612302780151, AUC: 0.8550099999999999


Test set: Avg. loss: 0.001117554724216461, AUC: 0.5081295


Test set: Avg. loss: 0.0006274791061878205, AUC: 0.8039589999999999


Test set: Avg. loss: 0.0006286765336990356, AUC: 0.827536


Test set: Avg. loss: 0.000628991961479187, AUC: 0.836644


Test set: Avg. loss: 0.0016907118558883666, AUC: 0.5814725


Test set: Avg. loss: 0.0007117418646812439, AUC: 0.8346469999999999


Test set: Avg. loss: 0.0007231528460979462, AUC: 0.8660589999999999



In [44]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = (col_names))

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [10]:
# 2 CLASS Focal Loss
# this code might not work 

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []

loss_fn_args = {}


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn=loss_fns.SigmoidFocalLoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["focal_loss", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.0012810109853744506, AUC: 0.4544585



TypeError: unsupported operand type(s) for *: 'Tensor' and 'NoneType'

In [38]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

NameError: name 'df2' is not defined

In [9]:
# distance + capped loss
momentum=0
learning_rates = [1e-3, 5e-3, 5e-4, 1e-4]


    
loss_fn_args = {}
loss_cap = 5.0
loss_fn_args['distance'] = 'euclidean'

start_epoch = 2

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNetWithEmbeddings(2)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(start_epoch):
            loss_fn_args['loss_cap'] = None
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
        for epoch in range(start_epoch, n_epochs + 1):
            loss_fn_args['loss_cap'] = loss_cap
            loss_fn_args['print_loss']=True
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)


for i in range(len(learning_rates)): 
    row = ["distance_capped_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], loss_fn_args['loss_cap'], norm]
    rows.append(row)


Test set: Avg. loss: 0.0006921598315238952, AUC: 0.5632135

Loss before cap
tensor([0.6831, 0.7459, 0.5364, 0.7497, 0.6382, 0.6398, 0.6786, 0.6400, 0.6401,
        0.7816, 0.5922, 0.6889, 0.8030, 0.6779, 0.7631, 0.6872, 0.5974, 0.6980,
        0.7272, 0.7339, 0.7323, 0.6067, 0.6251, 0.5210, 0.7205, 0.8327, 0.6379,
        0.6990, 0.6212, 0.4908, 0.7118, 0.6685, 0.5900, 0.6023, 0.5205, 0.7086,
        0.6193, 0.6757, 0.7100, 0.6086, 0.5149, 0.6846, 0.7233, 0.5548, 0.5982,
        0.5765, 0.6585, 0.7136, 0.7176, 0.6636, 0.6148, 0.5052, 0.7105, 0.6931,
        0.6034, 0.7542, 0.6064, 0.6969, 0.6078, 0.6080, 0.6383, 0.7433, 0.6470,
        0.5520], grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
SMOTE Labels
tensor([1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
        1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
        1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
        1., 0., 1., 0., 1., 1., 1., 0., 1., 

Loss before cap
tensor([0.7145, 0.6344, 0.5506, 0.7495, 0.7407, 0.6616, 0.6919, 0.7536, 0.6877,
        0.5829, 0.6430, 0.7394, 0.6798, 0.7321, 0.6201, 0.6801, 0.5896, 0.6265,
        0.6878, 0.6842, 0.6186, 0.8466, 0.7698, 0.7747, 0.8332, 0.6185, 0.7476,
        0.6019, 0.8067, 0.5933, 0.6862, 0.6779, 0.6503, 0.7643, 0.7107, 0.7879,
        0.7026, 0.6814, 0.6751, 0.7027, 0.5822, 0.7281, 0.8299, 0.7485, 0.7020,
        0.7527, 0.6714, 0.5699, 0.5841, 0.8051, 0.6592, 0.8382, 0.6943, 0.6130,
        0.6624, 0.6264, 0.7296, 0.6505, 0.6685, 0.7520, 0.6091, 0.6592, 0.5673,
        0.7971], grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
SMOTE Labels
tensor([0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0.,
        0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
        1., 1., 0., 1., 1., 0., 1., 1., 1., 0.], dtype=torch.float64)
Mask
[1. 1. 0. 1. 1. 0. 0. 1. 0. 0.

Mask
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Distance
tensor([1.5955, 1.2969,    nan,    nan,    nan,    nan,    nan, 1.0064,    nan,
        0.7462,    nan,    nan,    nan, 0.9320,    nan, 2.0256, 1.9194,    nan,
        1.4872, 2.0543,    nan,    nan, 0.9965, 1.1048, 1.0866, 1.0995, 1.6301,
           nan, 1.3789, 1.5019,    nan,    nan, 0.9799, 1.0817, 1.5295, 2.8884,
        0.9062,    nan, 1.9350, 0.9042, 1.1568, 2.4045,    nan,    nan, 0.9895,
        1.2165,    nan, 1.8904,    nan,    nan, 1.0604, 1.1203,    nan, 1.0451,
        0.9077,    nan, 0.9674, 1.8494, 2.5613,    nan, 1.2646, 2.0214,    nan,
        3.1216], grad_fn=<IndexPutBackward0>)
Loss before cap
tensor([0.6138, 0.5268, 0.6880, 0.7315, 0.6230, 0.6028, 0.6631, 0.6331, 0.7102,
        0.7182, 0.7450, 0.6643, 0.5758, 0.7928, 0.8095, 0.6030, 0.6774, 0.7261,
        

Loss before cap
tensor([0.6458, 0.6410, 0.7406, 0.7063, 0.6537, 0.5887, 0.6702, 0.5825, 0.5149,
        0.7292, 0.5692, 0.6561, 0.5797, 0.6868, 0.6796, 0.7380, 0.7566, 0.6767,
        0.6471, 0.4971, 0.6168, 0.6472, 0.5740, 0.7441, 0.7318, 0.5590, 0.7292,
        0.6963, 0.6422, 0.5792, 0.6838, 0.6461, 0.7455, 0.7506, 0.6693, 0.5473,
        0.6429, 0.6441, 0.7288, 0.7795, 0.6565, 0.6977, 0.5896, 0.7862, 0.6485,
        0.6075, 0.7351, 0.6552, 0.7069, 0.6095, 0.6823, 0.7597, 0.8440, 0.7639,
        0.7211, 0.5573, 0.7657, 0.6122, 0.7548, 0.6744, 0.8279, 0.7616, 0.7890,
        0.6711], grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
SMOTE Labels
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1.,
        1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1.,
        1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0.,
        1., 1., 0., 1., 0., 1., 0., 0., 0., 1.], dtype=torch.float64)
Mask
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.

Loss before cap
tensor([0.7469, 0.5831, 0.5732, 0.6479, 0.5905, 0.6276, 0.5340, 0.7392, 0.7687,
        0.6833, 0.7602, 0.6503, 0.7080, 0.5271, 0.7387, 0.6432, 0.6419, 0.7276,
        0.6422, 0.7341, 0.5969, 0.5588, 0.7364, 0.6953, 0.6954, 0.7255, 0.6641,
        0.6255, 0.7075, 0.7037, 0.6259, 0.5614, 0.5571, 0.6800, 0.7395, 0.6252,
        0.7295, 0.7647, 0.6470, 0.6822, 0.6471, 0.7291, 0.7688, 0.6844, 0.6045,
        0.7625, 0.4275, 0.7184, 0.7232, 0.7473, 0.6859, 0.7605, 0.7399, 0.7421,
        0.6608, 0.7069, 0.6620, 0.5786, 0.5767, 0.6101, 0.4848, 0.7421, 0.6519,
        0.6221], grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
SMOTE Labels
tensor([0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0.,
        1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
        0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 1., 1., 1., 1., 1., 0., 1., 1.], dtype=torch.float64)
Mask
[1. 0. 0. 0. 0. 0. 0. 1. 1. 0.

...

Loss before cap
tensor([0.5264, 0.6953, 0.5922, 0.7854, 0.5798, 0.5837, 0.6498, 0.6037, 0.6388,
        0.6893, 0.5843, 0.7004, 0.8698, 0.7127, 0.5392, 0.7087, 0.6388, 0.6114,
        0.8131, 0.5636, 0.6342, 0.7735, 0.6099, 0.6009, 0.7956, 0.6338, 0.6356,
        0.6997, 0.5981, 0.7137, 0.5775, 0.5953, 0.6853, 0.6570, 0.6934, 0.5769,
        0.7593, 0.7513, 0.6138, 0.6064, 0.5852, 0.6051, 0.6401, 0.6156, 0.7395,
        0.6215, 0.6380, 0.7510, 0.6183, 0.6070, 0.6905, 0.7379, 0.6294, 0.7497,
        0.6168, 0.5807, 0.6699, 0.7085, 0.7137, 0.6438, 0.7273, 0.7754, 0.5937,
        0.5177], grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
SMOTE Labels
tensor([1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
        0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
        0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
        1., 1., 0., 0., 0., 1., 0., 0., 1., 1.], dtype=torch.float64)
Mask
[0. 1. 0. 1. 0. 0. 0. 0. 0. 1.

Loss before cap
tensor([0.7119, 0.7780, 0.5780, 0.5498, 0.9141, 0.6790, 0.7296, 0.6370, 0.6121,
        0.6541, 0.5590, 0.5650, 0.6901, 0.6202, 0.6306, 0.7774, 0.7242, 0.5623,
        0.6011, 0.5615, 0.7529, 0.7555, 0.7911, 0.5658, 0.7359, 0.8008, 0.7342,
        0.7128, 0.8304, 0.7000, 0.7359, 0.6357, 0.7364, 0.7498, 0.5840, 0.5429,
        0.6293, 0.7001, 0.7521, 0.7480, 0.6904, 0.8151, 0.6010, 0.7732, 0.5622,
        0.6659, 0.6805, 0.5733, 0.6874, 0.7394, 0.6102, 0.7019, 0.6634, 0.7312,
        0.6763, 0.6575, 0.7574, 0.6755, 0.7847, 0.7372, 0.7589, 0.5637, 0.6453,
        0.7679], grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
SMOTE Labels
tensor([0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
        1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
        1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 1., 1., 0.], dtype=torch.float64)
Mask
[1. 1. 0. 0. 1. 0. 1. 0. 0. 0.

Loss before cap
tensor([0.7452, 0.7668, 0.6320, 0.7334, 0.7026, 0.6085, 0.6841, 0.7132, 0.7459,
        0.5934, 0.8191, 0.8322, 0.6325, 0.6801, 0.5397, 0.7497, 0.5969, 0.6297,
        0.7530, 0.6538, 0.6796, 0.5769, 0.6140, 0.5638, 0.7459, 0.5066, 0.6381,
        0.6558, 0.6952, 0.6134, 0.6952, 0.6209, 0.7424, 0.6761, 0.6856, 0.6062,
        0.7738, 0.7131, 0.6981, 0.7106, 0.7847, 0.7695, 0.6186, 0.7211, 0.5529,
        0.6929, 0.7347, 0.6405, 0.8081, 0.7004, 0.7631, 0.6371, 0.6938, 0.7255,
        0.7667, 0.6255, 0.7078, 0.6446, 0.6877, 0.6495, 0.7082, 0.7372, 0.6534,
        0.7396], grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
SMOTE Labels
tensor([0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
        0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 1., 1., 1., 0., 0., 1., 0.], dtype=torch.float64)
Mask
[1. 1. 0. 1. 1. 0. 1. 1. 1. 0.

Loss before cap
tensor([0.7709, 0.6721, 0.7480, 0.5557, 0.7172, 0.8102, 0.5077, 0.5995, 0.6562,
        0.7406, 0.7264, 0.5174, 0.7890, 0.7636, 0.5579, 0.7150, 0.6640, 0.6387,
        0.7328, 0.8401, 0.7294, 0.7157, 0.5634, 0.6481, 0.5322, 0.7172, 0.5146,
        0.6718, 0.4895, 0.6621, 0.6719, 0.6042, 0.8009, 0.7526, 0.6535, 0.6816,
        0.7598, 0.7673, 0.7440, 0.6534, 0.7452, 0.7290, 0.6888, 0.6454, 0.6952,
        0.6034, 0.7177, 0.5928, 0.6423, 0.7548, 0.6276, 0.7005, 0.5778, 0.7126,
        0.7189, 0.6796, 0.7445, 0.6992, 0.7870, 0.6349, 0.5714, 0.7122, 0.6540,
        0.7933], grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
SMOTE Labels
tensor([0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0.,
        0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
        0., 1., 0., 0., 0., 1., 1., 0., 0., 0.], dtype=torch.float64)
Mask
[1. 0. 1. 0. 1. 1. 0. 0. 0. 1.

KeyboardInterrupt: 

In [20]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [24]:
# distance + capped loss using whole class average tensor
momentum=0
learning_rates = [1e-3, 5e-3, 5e-4, 1e-4]


cap_aucs=[]

    
loss_fn_args = {}
loss_caps = [1, 5, 10]
loss_fn_args['distance'] = 'cosine'

start_epoch = 2

learning_rate_aucs = []

for loss_cap in loss_caps:
    
    learning_rate_aucs = []
    
    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                loss_fn_args['avg_tensors'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELossAvgDistance, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                loss_fn_args['print_loss']=False
                loss_fn_args['avg_tensors'] = []
                for k in range(2):
                    _, avg_tensor = network(avg_tensors_list[k])
                    loss_fn_args['avg_tensors'].append(avg_tensor)
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELossAvgDistance, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
        
    cap_aucs.append([auc_mean, auc_variance])

for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["cosine_distance_capped_smote_avg", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], loss_fn_args['loss_cap'], norm, None]
        rows.append(row)


Test set: Avg. loss: 0.0011167092323303223, AUC: 0.386697


Test set: Avg. loss: 0.0006031518578529358, AUC: 0.858937


Test set: Avg. loss: 0.0007109511196613312, AUC: 0.7610769999999999


Test set: Avg. loss: 0.0006870960295200347, AUC: 0.8417955


Test set: Avg. loss: 0.001104489803314209, AUC: 0.600618


Test set: Avg. loss: 0.000642341673374176, AUC: 0.8279484999999999


Test set: Avg. loss: 0.0006148490905761719, AUC: 0.850252


Test set: Avg. loss: 0.000648755669593811, AUC: 0.8505670000000001


Test set: Avg. loss: 0.003014646053314209, AUC: 0.291365


Test set: Avg. loss: 0.0004923479855060577, AUC: 0.879308


Test set: Avg. loss: 0.0006954239904880524, AUC: 0.875877


Test set: Avg. loss: 0.0011900174617767334, AUC: 0.828035


Test set: Avg. loss: 0.005821780920028686, AUC: 0.2825655


Test set: Avg. loss: 0.0006962125897407532, AUC: 0.5029950000000001


Test set: Avg. loss: 0.0007013527750968933, AUC: 0.500999


Test set: Avg. loss: 0.0006328832805156708, AUC: 0.84173499999


Test set: Avg. loss: 0.004506844282150269, AUC: 0.4027195


Test set: Avg. loss: 0.0006527964770793915, AUC: 0.7828725


Test set: Avg. loss: 0.0006425573527812958, AUC: 0.8165255000000001


Test set: Avg. loss: 0.0006454679667949677, AUC: 0.8199185000000002


Test set: Avg. loss: 0.0018708327412605287, AUC: 0.7087910000000002


Test set: Avg. loss: 0.0006200909316539764, AUC: 0.8200980000000001


Test set: Avg. loss: 0.00060707026720047, AUC: 0.8420890000000001


Test set: Avg. loss: 0.0006036153733730317, AUC: 0.8501780000000001


Test set: Avg. loss: 0.0018799042701721192, AUC: 0.435471


Test set: Avg. loss: 0.0006488969027996063, AUC: 0.7791954999999999


Test set: Avg. loss: 0.0006456703245639801, AUC: 0.7969269999999999


Test set: Avg. loss: 0.0006377524137496948, AUC: 0.8093465


Test set: Avg. loss: 0.0030028048753738402, AUC: 0.6811449999999999


Test set: Avg. loss: 0.0006707220077514648, AUC: 0.7317315


Test set: Avg. loss: 0.0006651672124862671, AUC: 0.7647250000000001



Test set: Avg. loss: 0.0005985681712627411, AUC: 0.821609


Test set: Avg. loss: 0.0006865729093551635, AUC: 0.7165375


Test set: Avg. loss: 0.0008287043273448944, AUC: 0.524346


Test set: Avg. loss: 0.000615205317735672, AUC: 0.8489694999999999


Test set: Avg. loss: 0.0005668532848358154, AUC: 0.880726


Test set: Avg. loss: 0.0006448164582252503, AUC: 0.834132


Test set: Avg. loss: 0.002872795581817627, AUC: 0.682523


Test set: Avg. loss: 0.0006151713728904724, AUC: 0.848335


Test set: Avg. loss: 0.000639518678188324, AUC: 0.881957


Test set: Avg. loss: 0.0005816068947315216, AUC: 0.893872


Test set: Avg. loss: 0.002558753967285156, AUC: 0.509605


Test set: Avg. loss: 0.0006600413918495178, AUC: 0.7879510000000003


Test set: Avg. loss: 0.0006359151005744934, AUC: 0.834866


Test set: Avg. loss: 0.0006491739749908447, AUC: 0.8294060000000001


Test set: Avg. loss: 0.002579985499382019, AUC: 0.547603


Test set: Avg. loss: 0.0006696692407131195, AUC: 0.752434


Test set: Avg


Test set: Avg. loss: 0.000771642655134201, AUC: 0.7791220000000001


Test set: Avg. loss: 0.0007035957872867585, AUC: 0.8487339999999999


Test set: Avg. loss: 0.0009450371563434601, AUC: 0.8339224999999999


Test set: Avg. loss: 0.001141647696495056, AUC: 0.38259699999999996


Test set: Avg. loss: 0.000636038988828659, AUC: 0.8309105


Test set: Avg. loss: 0.000625913679599762, AUC: 0.8521540000000001


Test set: Avg. loss: 0.0008344018459320069, AUC: 0.861994


Test set: Avg. loss: 0.002030069828033447, AUC: 0.4423475


Test set: Avg. loss: 0.0006735901832580567, AUC: 0.8416589999999999


Test set: Avg. loss: 0.0008218474090099335, AUC: 0.8355880000000001


Test set: Avg. loss: 0.0010065958201885222, AUC: 0.8438029999999999


Test set: Avg. loss: 0.0007690939903259277, AUC: 0.342897


Test set: Avg. loss: 0.0006489973664283753, AUC: 0.8481580000000001


Test set: Avg. loss: 0.0008452142179012299, AUC: 0.770876


Test set: Avg. loss: 0.0007234020829200745, AUC: 0.8641479999999999


T

In [25]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [31]:
# cosine distance + capped loss
momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]

cap_aucs = []
    
loss_fn_args = {}
loss_caps = [0.5]
loss_fn_args['distance'] = 'cosine'


start_epoch = 1


for loss_cap in loss_caps: 
    
    learning_rate_aucs = []
    
    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['print_capped'] = False
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['print_capped'] = False
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    cap_aucs.append([auc_mean, auc_variance])

for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["cosine_distance_capped_smote", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], loss_fn_args['loss_cap'], norm, None]
        rows.append(row)


Test set: Avg. loss: 0.0008583230078220368, AUC: 0.43444499999999997


Test set: Avg. loss: 0.0005388941168785095, AUC: 0.8616320000000001


Test set: Avg. loss: 0.0005308660864830017, AUC: 0.866123


Test set: Avg. loss: 0.0004951866567134857, AUC: 0.877475


Test set: Avg. loss: 0.0013088712692260742, AUC: 0.5226230000000001


Test set: Avg. loss: 0.0006043859422206879, AUC: 0.855531


Test set: Avg. loss: 0.0014131583571434022, AUC: 0.8066570000000001


Test set: Avg. loss: 0.0006903581321239472, AUC: 0.889435


Test set: Avg. loss: 0.00092867112159729, AUC: 0.46022599999999997


Test set: Avg. loss: 0.0006813707053661347, AUC: 0.6800725


Test set: Avg. loss: 0.0006745235919952393, AUC: 0.7929700000000001


Test set: Avg. loss: 0.0007089492678642273, AUC: 0.7771460000000001


Test set: Avg. loss: 0.002867063760757446, AUC: 0.368242


Test set: Avg. loss: 0.0006618314683437348, AUC: 0.8063965


Test set: Avg. loss: 0.0007057085931301117, AUC: 0.8448909999999998


Test set: Avg. los

In [32]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [8]:
# 2 class triplet loss no ratio 
# no smote 


momentum=0
learning_rates = [(5e-6, 1e-3), (1e-6, 5e-4), (1e-6, 1e-4), (5e-6, 5e-4)]

learning_rate_aucs = []

    

for learning_rate in learning_rates:
    aucs = []
    for i in range(10): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddings(2)
        linear_probe = models.ConvNetLinearProbe(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss(epoch, train_loader_tripletloss, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
            if (epoch + 1) % 10 == 0: 
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)
    
learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)
    
    



for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], 0.0, norm]
    rows.append(row)


Test set: Avg. loss: 0.004998538017272949, AUC: 0.5778245



RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []], which is output 0 of ReluBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [9]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [12]:
# 2 class triplet loss with capped SMOTE 

# note: sometimes can get a very high accuracy but may diverge

momentum=0
learning_rates = [(5e-6, 1e-3), (1e-6, 5e-4), (1e-6, 1e-4), (5e-6, 5e-4)]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = 5.0
loss_fn_args['distance'] = 'euclidean'


for learning_rate in learning_rates:
    aucs = []
    for i in range(10): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddings(2)
        linear_probe = models.ConvNetLinearProbe(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss_smote(epoch, train_loader_tripletloss_smote, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
            if (epoch + 1) % 10 == 0: 
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)



for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], 0.0, norm]
    rows.append(row)


Test set: Avg. loss: 0.0006959781944751739, AUC: 0.44473399999999996



RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []], which is output 0 of ReluBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [11]:
# 2 class triplet loss with ratio 
# no smote 

momentum=0
learning_rates = [(1e-7, 1e-5)]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(3): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddings(2)
        linear_probe = models.ConvNetLinearProbe(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss(epoch, train_loader_tripletloss_ratio, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(50):
            _, _ = train.train_linear_probe(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
            if (epoch + 1) % 10 == 0: 
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], 
           auc_mean[i][4], auc_variance[i][4],
           auc_mean[i][5], auc_variance[i][5],
           None]
    rows.append(row)


Test set: Avg. loss: 0.0007769797742366791, AUC: 0.581019

Train loss: 16.096790712090986
Train loss: 7.95101783849016
Train loss: 1.6742434924161886
Train loss: 0.0
Train loss: 6.369154097158698
Train loss: 1.0685905746266813
Train loss: 0.9919730077815961
Train loss: 4.3510085480122624
Train loss: 0.0
Train loss: 0.0
Train loss: 0.0
Train loss: 1.729099128819719
Train loss: 0.0
Train loss: 2.9074361294130737
Train loss: 0.0

Test set: Avg. loss: 0.0007081505060195923, AUC: 0.595766


Test set: Avg. loss: 0.0006918745040893554, AUC: 0.599685


Test set: Avg. loss: 0.0006822091042995453, AUC: 0.606827


Test set: Avg. loss: 0.0006758022308349609, AUC: 0.614831


Test set: Avg. loss: 0.0006717154681682586, AUC: 0.620797


Test set: Avg. loss: 0.0006670982539653778, AUC: 0.6299465


Test set: Avg. loss: 0.0006633270978927612, AUC: 0.641555


Test set: Avg. loss: 0.0006615970730781555, AUC: 0.647514


Test set: Avg. loss: 0.0006594435572624207, AUC: 0.6563729999999999


Test set: Avg. lo


Test set: Avg. loss: 0.0006901566684246063, AUC: 0.5230110000000001


Test set: Avg. loss: 0.0006899235546588897, AUC: 0.5261325


Test set: Avg. loss: 0.0006896623373031616, AUC: 0.5253125


Test set: Avg. loss: 0.0006894396543502808, AUC: 0.5309490000000001


Test set: Avg. loss: 0.0006897644698619842, AUC: 0.52842


Test set: Avg. loss: 0.0006899001598358154, AUC: 0.525083


Test set: Avg. loss: 0.0006898137331008911, AUC: 0.5242764999999999


Test set: Avg. loss: 0.0006899007260799408, AUC: 0.5234755


Test set: Avg. loss: 0.0006897071897983551, AUC: 0.5284414999999999


Test set: Avg. loss: 0.0006894522905349732, AUC: 0.5316019999999999


Test set: Avg. loss: 0.0006894703507423401, AUC: 0.5321385000000001


Test set: Avg. loss: 0.0006892255246639252, AUC: 0.5359525


Test set: Avg. loss: 0.0006892721951007843, AUC: 0.5328990000000001


Test set: Avg. loss: 0.0006891983449459076, AUC: 0.537638


Test set: Avg. loss: 0.0006890941560268402, AUC: 0.5369265000000001


Test set: Avg. l

In [12]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

ValueError: 14 columns passed, passed data had 18 columns

In [31]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [8]:
# capped smote using triplet loss

momentum=0
learning_rates = [5e-3, 1e-3, 1e-2]

loss_fn_args = {}
loss_fn_args['print_loss']=False


cap_aucs = []

start_epoch = 5

loss_caps = [1, 5, 10]

for loss_cap in loss_caps:
    
    learning_rate_aucs = []
    
    for learning_rate in learning_rates:
        aucs = []
        for i in range(10): 
            print(loss_cap)
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=loss_fns.TripletLoss,loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args, print_dist=True)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)
    
    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
        
    cap_aucs.append([auc_mean, auc_variance])



for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm, "margin=0.5"]
        rows.append(row)

1

Test set: Avg. loss: 0.0006921598315238952, AUC: 0.5632135

Triplet Loss Calculation
tensor([1.0000e-05, 1.9810e-01, 1.5458e+00, 1.6848e+01, 1.5036e+01, 1.0000e-05,
        2.2834e-01, 6.2688e+00, 1.0000e-05, 3.2163e+00, 1.0000e-05, 7.8707e+00,
        7.7019e+00, 6.4259e+00, 6.5882e+00, 2.3989e+01, 2.4655e-01, 5.3974e+01,
        1.5999e+01, 1.1811e+00, 2.1506e+01, 2.7749e+00, 1.0000e-05, 3.0049e+00,
        1.0000e-05, 3.4720e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 5.6800e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.4092e+02, 1.0000e-05, 1.6968e+01, 2.8015e+00, 7.9793e-01, 2.9657e+00,
        6.2538e-01, 1.0000e-05, 2.3499e+01, 2.4282e+01, 6.7770e+01, 1.0000e-05,
        9.8537e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.4370e+01, 5.5514e+00,
        1.0000e-05, 1.0000e-05, 1.0707e+01, 1.0000e-05, 6.6199e+00, 1.0000e-05,
        1.0000e-05, 1.9218e+01, 1.0000e-05, 1.0000e-05],
       grad_fn=<IndexPutBackward0>)
CAP ARRAY
Triplet L

Triplet Loss Calculation
tensor([8.1309e+01, 1.0000e-05, 3.2372e+01, 3.4662e+01, 1.0000e-05, 1.4970e+01,
        9.5361e+01, 6.7757e+01, 8.4828e+01, 1.0000e-05, 4.7372e+01, 1.0000e-05,
        6.9025e-01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        3.0591e+01, 4.6644e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 5.9640e+01, 1.1249e+01, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 2.6429e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 7.3334e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 8.5215e-01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 4.8436e+01,
        3.9171e+00, 1.1749e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        2.1211e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 8.2269e+00, 1.8661e+00,
        1.0000e-05, 1.8646e+02, 2.8457e+00, 1.0000e-05],
       grad_fn=<IndexPutBackward0>)
CAP ARRAY
Triplet Loss Calculation
tensor([1.0000e-05, 3.7766e-02, 1.0000e-05, 1.1

Triplet Loss Calculation
tensor([6.2957e+01, 1.0000e-05, 1.0000e-05, 5.2106e-01, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 2.5816e+01, 8.4301e+00,
        7.3015e+01, 1.0000e-05, 1.6642e+01, 1.0000e-05, 1.0000e-05, 4.3282e+01,
        1.4071e+01, 1.1287e+01, 2.9678e+01, 1.0000e-05, 3.5300e+01, 1.0000e-05,
        4.2568e+00, 4.8654e+00, 1.0309e+02, 1.4843e+01, 1.0000e-05, 1.0000e-05,
        3.7864e+00, 1.0000e-05, 1.0000e-05, 8.6994e+00, 4.6624e-01, 5.2929e+01,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.1371e+01,
        1.1258e+02, 6.6859e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        2.1986e+01, 2.1362e+00, 1.3331e+02, 1.1181e+00, 3.7818e+00, 1.0000e-05,
        1.0000e-05, 6.0909e+00, 1.9902e+00, 1.5457e+02],
       grad_fn=<IndexPutBackward0>)
CAP ARRAY
Triplet Loss Calculation
tensor([1.0000e-05, 1.0000e-05, 1.0000e-05, 4.1

Triplet Loss Calculation
tensor([1.0000e-05, 1.3476e+01, 2.1310e+01, 1.0000e-05, 1.2216e+01, 1.0000e-05,
        2.1944e+01, 6.1457e+00, 7.9575e+00, 1.0000e-05, 1.3725e+01, 1.0000e-05,
        8.1142e+00, 1.0000e-05, 1.1451e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 6.4693e+00, 1.0000e-05, 1.0000e-05, 4.0372e+01,
        1.0000e-05, 1.0000e-05, 1.5488e+01, 1.0000e-05, 1.0000e-05, 4.1650e+01,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.9016e+01, 7.1184e-01,
        1.0000e-05, 1.0000e-05, 1.7127e+00, 9.4128e+00, 2.6249e+01, 1.0000e-05,
        7.8826e+01, 3.0626e+01, 7.8392e+00, 1.0000e-05, 1.2451e+01, 1.0000e-05,
        4.0367e+00, 1.0616e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 8.8239e-01,
        3.7133e+00, 8.3462e+00, 1.9987e+01, 1.0000e-05, 1.5906e+02, 1.0000e-05,
        1.0000e-05, 6.7304e+00, 4.7341e+01, 1.0000e-05],
       grad_fn=<IndexPutBackward0>)
CAP ARRAY
Triplet Loss Calculation
tensor([1.0000e-05, 1.0000e-05, 1.0000e-05, 2.1

Triplet Loss Calculation
tensor([1.3114e+02, 1.0000e-05, 2.8383e+00, 3.8215e+00, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.1780e+01, 1.6594e+02, 2.8124e+01,
        3.7852e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.4877e+01, 2.8342e+01, 6.0890e-01, 7.6770e-01, 1.0000e-05, 1.0000e-05,
        3.2925e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.6102e+02, 1.0000e-05, 1.0000e-05, 4.4224e+01, 3.4084e+00,
        3.1044e+01, 1.0000e-05, 1.0000e-05, 2.4473e+00, 5.1023e+00, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 1.5014e+01, 1.0000e-05, 5.3993e+00, 2.7245e+01,
        1.0000e-05, 2.9755e+00, 1.0000e-05, 5.3099e+00, 1.0000e-05, 4.9329e+00,
        1.0000e-05, 9.2819e+01, 1.0000e-05, 1.8710e+00, 1.0000e-05, 1.0000e-05,
        2.5021e+01, 7.5605e+01, 3.9480e-01, 1.0000e-05],
       grad_fn=<IndexPutBackward0>)
CAP ARRAY
Triplet Loss Calculation
tensor([1.0000e-05, 2.2226e+00, 2.9219e-02, 1.2

...

Triplet Loss Calculation
tensor([4.7399e+01, 3.7551e+00, 4.0964e+02, 5.4642e+00, 1.6723e+00, 1.8553e+02,
        1.0000e-05, 1.3158e+02, 3.0866e+01, 1.0000e-05, 3.3094e+01, 1.0000e-05,
        4.1827e+01, 9.3571e+01, 1.5994e+00, 1.0000e-05, 1.2590e+02, 1.0000e-05,
        6.6315e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        4.5046e+01, 5.1913e+01, 1.0000e-05, 1.0000e-05, 4.3779e+00, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 2.0535e+00, 1.0000e-05, 5.1981e+01, 6.1463e+00,
        1.0000e-05, 4.7935e+00, 1.5304e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.7038e+01, 2.3217e+02, 1.0000e-05,
        1.0000e-05, 5.1166e+00, 1.0853e+01, 7.9116e+00, 1.0000e-05, 1.6826e+00,
        2.0102e+00, 2.3248e+00, 5.7142e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        3.5377e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05],
       grad_fn=<IndexPutBackward0>)
CAP ARRAY
Triplet Loss Calculation
tensor([3.9816e+00, 1.0000e-05, 1.3854e+01, 1.0

Triplet Loss Calculation
tensor([1.0898e+01, 1.0000e-05, 3.1477e+01, 1.0000e-05, 1.0000e-05, 1.6543e+01,
        5.1291e+00, 8.4552e-01, 1.0000e-05, 9.0497e+01, 1.0000e-05, 1.2182e+02,
        1.0000e-05, 8.3125e+00, 2.6412e+01, 3.0136e+01, 1.0000e-05, 7.6907e+01,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.1242e+00, 1.0000e-05, 1.0000e-05,
        6.3705e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.9548e+02, 7.4929e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        2.3026e+02, 1.0000e-05, 1.6138e-01, 2.0665e+00, 8.5308e+01, 1.0000e-05,
        2.6639e+01, 1.0000e-05, 1.0000e-05, 5.2334e+01, 1.0000e-05, 1.0000e-05,
        3.9900e+00, 1.0000e-05, 9.3299e+00, 1.0000e-05, 1.3057e+00, 4.5736e+00,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0112e+01, 1.0658e+00,
        1.2689e+01, 1.0000e-05, 1.0000e-05, 4.4302e+01],
       grad_fn=<IndexPutBackward0>)
CAP ARRAY
Triplet Loss Calculation
tensor([8.9483e-01, 1.0000e-05, 1.0000e-05, 1.0

Triplet Loss Calculation
tensor([5.8289e+00, 1.0000e-05, 1.0000e-05, 2.8580e+01, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 2.0079e+00, 2.1275e+00, 1.0000e-05, 1.0000e-05,
        4.3033e+01, 7.4919e-01, 1.0000e-05, 2.7836e+01, 2.4359e+01, 1.0000e-05,
        1.2296e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 6.9946e+00,
        3.6318e+01, 1.0000e-05, 1.0000e-05, 4.1951e+00, 2.8090e+00, 1.0000e-05,
        1.9243e+00, 1.1971e+00, 1.0000e-05, 5.3540e+01, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.2537e+01, 9.2674e+01, 1.0000e-05, 8.8826e+00, 2.0657e+01,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        3.5927e-01, 1.3172e+02, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 1.5735e+01, 1.0000e-05, 2.6279e+01, 1.5404e+01,
        1.7860e+01, 7.4034e+00, 7.2934e+00, 1.3190e+01],
       grad_fn=<IndexPutBackward0>)
CAP ARRAY
Triplet Loss Calculation
tensor([1.1162e+00, 8.2692e+00, 3.4450e+00, 1.0

Triplet Loss Calculation
tensor([8.8794e+00, 1.0000e-05, 1.0000e-05, 9.5122e-01, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 2.1000e+02, 1.0000e-05, 1.0000e-05, 1.5998e+01, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 3.9132e+01, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 8.8608e+01, 3.1843e+01, 1.0000e-05,
        3.1778e+01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 6.3716e-01, 1.3948e+02, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        7.8348e+01, 1.0000e-05, 1.0000e-05, 2.2640e+00, 3.3527e+01, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        7.0931e+01, 1.0000e-05, 7.6804e+00, 3.8084e-02, 4.2671e+01, 1.4026e-01,
        4.3345e+01, 1.0000e-05, 7.1119e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.1352e+01, 1.8283e+01, 5.9960e-01],
       grad_fn=<IndexPutBackward0>)
CAP ARRAY
Triplet Loss Calculation
tensor([4.0058e+01, 1.0000e-05, 1.0000e-05, 1.0

KeyboardInterrupt: 

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [52]:
# capped smote using triplet loss w/ average 

momentum=0
learning_rates = [5e-3, 1e-3, 1e-2]

loss_fn_args = {}
cap_aucs = []

start_epoch = 2

loss_caps = [1, 5, 10]
cap_calc = loss_fns.TripletLossWithAverage

for loss_cap in loss_caps:
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10): 
            print(loss_cap)
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=cap_calc, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)
        
    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
        
    cap_aucs.append([auc_mean, auc_variance])


for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote_average", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

1

Test set: Avg. loss: 0.0006964887678623199, AUC: 0.3482715


Test set: Avg. loss: 0.00073517307639122, AUC: 0.841398


Test set: Avg. loss: 0.0010314007997512817, AUC: 0.826527


Test set: Avg. loss: 0.0009717654287815094, AUC: 0.8537300000000001

1

Test set: Avg. loss: 0.0006961984634399415, AUC: 0.47539049999999994


Test set: Avg. loss: 0.0007382735610008239, AUC: 0.825611


Test set: Avg. loss: 0.0009567088186740875, AUC: 0.831588


Test set: Avg. loss: 0.0015623956918716432, AUC: 0.8068925

1

Test set: Avg. loss: 0.0006931769251823425, AUC: 0.509471


Test set: Avg. loss: 0.0006451610028743743, AUC: 0.8464630000000001


Test set: Avg. loss: 0.0012270017266273498, AUC: 0.8370259999999999


Test set: Avg. loss: 0.0013078323006629944, AUC: 0.864949

1

Test set: Avg. loss: 0.0006944571435451508, AUC: 0.445826


Test set: Avg. loss: 0.0008669923841953278, AUC: 0.8259850000000001


Test set: Avg. loss: 0.0009103761315345764, AUC: 0.8520369999999999


Test set: Avg. loss: 0.0009062

5

Test set: Avg. loss: 0.0006908078789710999, AUC: 0.6734819999999999


Test set: Avg. loss: 0.0006440731883049011, AUC: 0.8425530000000001


Test set: Avg. loss: 0.001008029878139496, AUC: 0.8353280000000001


Test set: Avg. loss: 0.0011021165251731873, AUC: 0.8578000000000001

5

Test set: Avg. loss: 0.0006938815414905548, AUC: 0.4701365


Test set: Avg. loss: 0.0006398897767066955, AUC: 0.823039


Test set: Avg. loss: 0.0009389999508857727, AUC: 0.8418840000000001


Test set: Avg. loss: 0.0012309846878051757, AUC: 0.839351

5

Test set: Avg. loss: 0.0006928956806659699, AUC: 0.5152815


Test set: Avg. loss: 0.0006695924997329711, AUC: 0.836996


Test set: Avg. loss: 0.000713811069726944, AUC: 0.869234


Test set: Avg. loss: 0.0009120012223720551, AUC: 0.884378

5

Test set: Avg. loss: 0.0006952646076679229, AUC: 0.454125


Test set: Avg. loss: 0.0007137651145458221, AUC: 0.819804


Test set: Avg. loss: 0.0005585785508155823, AUC: 0.794235


Test set: Avg. loss: 0.001033907383680343


Test set: Avg. loss: 0.0008677043914794922, AUC: 0.873766

10

Test set: Avg. loss: 0.0006948215365409851, AUC: 0.44974300000000006


Test set: Avg. loss: 0.0006209433376789094, AUC: 0.859944


Test set: Avg. loss: 0.0011180386543273926, AUC: 0.8403849999999999


Test set: Avg. loss: 0.0008133378028869629, AUC: 0.876189

10

Test set: Avg. loss: 0.0006891251504421235, AUC: 0.614881


Test set: Avg. loss: 0.0007206940054893493, AUC: 0.822441


Test set: Avg. loss: 0.000845925122499466, AUC: 0.8559945000000001


Test set: Avg. loss: 0.0010710387229919433, AUC: 0.867974

10

Test set: Avg. loss: 0.0006960428357124329, AUC: 0.439736


Test set: Avg. loss: 0.0005857323706150055, AUC: 0.841833


Test set: Avg. loss: 0.0009211321473121643, AUC: 0.846067


Test set: Avg. loss: 0.0010535757541656494, AUC: 0.8533420000000002

10

Test set: Avg. loss: 0.0006958933174610138, AUC: 0.37558400000000003


Test set: Avg. loss: 0.0007239218056201935, AUC: 0.830268


Test set: Avg. loss: 0.0007279396951

In [53]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [54]:
# capped smote using triplet loss w/ positive dist squared 

momentum=0
learning_rates = [5e-3, 1e-3, 1e-2]

loss_fn_args = {}


start_epoch = 2

loss_caps = [1, 5, 10]
cap_calc = loss_fns.TripletLoss

cap_aucs = []


for loss_cap in loss_caps:
    learning_rate_aucs = []
    for learning_rate in learning_rates:
        aucs = []
        for i in range(10): 
            print(loss_cap)
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=cap_calc, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)


    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    cap_aucs.append([auc_mean, auc_variance])



for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote_pos_squared", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

1

Test set: Avg. loss: 0.0006953137814998627, AUC: 0.41351000000000004


Test set: Avg. loss: 0.0012100113034248352, AUC: 0.751849


Test set: Avg. loss: 0.001347662389278412, AUC: 0.779907


Test set: Avg. loss: 0.0013431642651557923, AUC: 0.8068489999999999

1

Test set: Avg. loss: 0.000698688566684723, AUC: 0.35026250000000003


Test set: Avg. loss: 0.0011922909617424011, AUC: 0.7699495000000001


Test set: Avg. loss: 0.001296019196510315, AUC: 0.8160130000000001


Test set: Avg. loss: 0.0014232771992683411, AUC: 0.8320620000000001

1

Test set: Avg. loss: 0.0006938562393188477, AUC: 0.47384050000000005


Test set: Avg. loss: 0.0011157945990562439, AUC: 0.7417819999999999


Test set: Avg. loss: 0.0013826609253883362, AUC: 0.776867


Test set: Avg. loss: 0.0012448267340660095, AUC: 0.806332

1

Test set: Avg. loss: 0.000687132328748703, AUC: 0.732907


Test set: Avg. loss: 0.0013641536831855774, AUC: 0.7088535000000001


Test set: Avg. loss: 0.0014020829200744628, AUC: 0.75314799999


Test set: Avg. loss: 0.0012890581488609314, AUC: 0.818852

5

Test set: Avg. loss: 0.0006957947611808777, AUC: 0.4403390000000001


Test set: Avg. loss: 0.0013087846636772155, AUC: 0.7569419999999999


Test set: Avg. loss: 0.0014033986926078796, AUC: 0.7918499999999999


Test set: Avg. loss: 0.0011560410261154176, AUC: 0.82701

5

Test set: Avg. loss: 0.0006878388524055481, AUC: 0.7391975


Test set: Avg. loss: 0.001277011215686798, AUC: 0.733312


Test set: Avg. loss: 0.0013133590817451476, AUC: 0.7746850000000001


Test set: Avg. loss: 0.0013910502791404725, AUC: 0.7918069999999999

5

Test set: Avg. loss: 0.0006964699029922485, AUC: 0.3996565000000001


Test set: Avg. loss: 0.001165420174598694, AUC: 0.743013


Test set: Avg. loss: 0.0013641301393508911, AUC: 0.789986


Test set: Avg. loss: 0.0014104530215263366, AUC: 0.8106280000000001

5

Test set: Avg. loss: 0.0006948442161083221, AUC: 0.46871799999999997


Test set: Avg. loss: 0.0012675276398658752, AUC: 0.740739


Test set: Av

10

Test set: Avg. loss: 0.0006926549971103668, AUC: 0.532581


Test set: Avg. loss: 0.0011874104738235473, AUC: 0.76892


Test set: Avg. loss: 0.0012728224992752076, AUC: 0.8023020000000001


Test set: Avg. loss: 0.0012980905175209046, AUC: 0.826338

10

Test set: Avg. loss: 0.0007008664906024933, AUC: 0.3223985


Test set: Avg. loss: 0.0012494014501571655, AUC: 0.74944


Test set: Avg. loss: 0.0012630877494812013, AUC: 0.7997829999999999


Test set: Avg. loss: 0.0013315749168395995, AUC: 0.8174549999999999

10

Test set: Avg. loss: 0.0006991316080093384, AUC: 0.33853449999999996


Test set: Avg. loss: 0.0011583274602890014, AUC: 0.7944675000000001


Test set: Avg. loss: 0.0012185652256011964, AUC: 0.824557


Test set: Avg. loss: 0.001334098756313324, AUC: 0.8564489999999999

10

Test set: Avg. loss: 0.0006847937405109406, AUC: 0.705627


Test set: Avg. loss: 0.0011758599281311035, AUC: 0.7457389999999999


Test set: Avg. loss: 0.0012690225839614869, AUC: 0.793452


Test set: Avg. los

In [55]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []